# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use variables defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from HW4 (theory part). 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # Create a copy of the graph to avoid modifying the original graph
    flow_graph = graph.copy()
    source_demand = 0 #for checking feasibility
    sink_demand = 0

    # Add a supersource and supersink to handle the demands
    supersource = 'supersource'
    supersink = 'supersink'
    flow_graph.add_node(supersource)
    flow_graph.add_node(supersink)

    # Connect supersource to the source nodes and the supersink to the sink nodes
    for state in flow_graph.nodes():
        demand = flow_graph.nodes[state].get('demand', 0) #For retirieving the demand, if it is not present it defaluts to zero
        if demand < 0:
            flow_graph.add_edge(supersource, state, capacity=abs(demand)) #Connecting the source to a supersource with the demand values
            source_demand +=  abs(demand)
            
        elif demand > 0:
            flow_graph.add_edge(state, supersink, capacity=abs(demand)) #Connecting the sinks to the supersink with the demand values
            sink_demand +=  abs(demand)
            

    # Compute the maximum flow
    flow_value, flow_dict = nx.maximum_flow(flow_graph, supersource, supersink ) #getting the flow value and the flow dictionary from the flow graph

    #Checking if max_flow is feasible
    #1.For a flow network with demands net demands should be conserved i.e. sum(demands)=0
    if sink_demand != source_demand:
        raise nx.NetworkXUnfeasible("No flow satisfies the demands.")
    #2.Necessary conditions for maxflow : flow into a node = flow out of a node
    for node in flow_graph.nodes():
        if node != supersink and node !=supersource: #this statement is true for all nodes except the sink node and the source node
            demand = flow_graph.nodes[node].get('demand', 0)

        # Calculate net flow into the node
            net_flow = sum(flow_dict[other_node][node] for other_node in graph.predecessors(node)) - \
                        sum(flow_dict[node][other_node] for other_node in graph.successors(node))

        # Check if net flow doesn't match the demand
            if demand != net_flow:
                raise nx.NetworkXUnfeasible("No flow satisfies the demands.")
    #3. The flowvalue in a flow with demands should be equal to the source demand
    if source_demand != flow_value:
        raise nx.NetworkXUnfeasible("No flow satisfier the demands")


    # Remove the added edges and nodes from the flow_dict
    del flow_dict[supersource]
    del flow_dict[supersink]
    for node in list(flow_dict.keys()):
        if supersource in flow_dict[node]:
            del flow_dict[node][supersource]

        if supersink in flow_dict[node]:
            del flow_dict[node][supersink]

    return flow_dict





To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    #Using Dictionary to get the nodes i.e. keys
    net_flow = {node: 0 for node in flow.keys()}

    #Using Dictionary to compute the net_flow out of each node
    for s, t in flow.items():
        for t, flow_value in t.items():
            net_flow[s] -= flow_value
            net_flow[t] += flow_value

    return net_flow


The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
# flow = flow_with_demands(G)
# div = divergence(flow)
# print ("Flow satisfies all demands:", all(div[n] == G.nodes[n]['demand'] for n in G.nodes()))

Flow satisfies all demands: True
